<a href="https://colab.research.google.com/github/kooose38/create_model/blob/dev/pytorch_lightning_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optunaによるパラメータの最適化


In [1]:
!pip install -q pytorch_lightning

     |████████████████████████████████| 829kB 5.8MB/s 
     |████████████████████████████████| 112kB 31.0MB/s 
     |████████████████████████████████| 829kB 22.8MB/s 
     |████████████████████████████████| 276kB 39.9MB/s 
     |████████████████████████████████| 1.3MB 32.5MB/s 
     |████████████████████████████████| 143kB 39.8MB/s 
     |████████████████████████████████| 296kB 43.0MB/s 


In [2]:
!pip install -q optuna

     |████████████████████████████████| 296kB 4.3MB/s 
     |████████████████████████████████| 163kB 31.2MB/s 
     |████████████████████████████████| 81kB 8.4MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 112kB 32.1MB/s 
     |████████████████████████████████| 143kB 26.9MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 


### Pytorch_lightningのleening-rate,中間層の最適化に使用する。

In [26]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import pandas as pd 
import optuna 
from sklearn.datasets import load_breast_cancer
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy
from pytorch_lightning.callbacks import EarlyStopping

In [9]:
cancer = load_breast_cancer()
x=cancer.data
t=cancer.target

In [10]:
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)


In [12]:
dataset = torch.utils.data.TensorDataset(x, t)
dataset

In [13]:
n_train = int(len(dataset)*0.6)
n_val = int(len(dataset)*0.2)
n_test = len(dataset) - n_train - n_val

torch.manual_seed(0)
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

batch_size=32

train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True, drop_last=True)


### ハイパーパラメータ
1. 中間層の数
2. 中間層のノード数
3. Learning Rate

In [56]:
class Net(pl.LightningModule):
  def __init__(self, lr=1e-1, n_mid=10, n_layers=1):
    super().__init__()
    self.n_mid = n_mid
    self.n_layers = n_layers
    self.lr = lr
    #中間ノードの数を最適化
    self.layers = nn.Sequential()
    for i in range(n_layers):
      if i == 0:
        self.layers.add_module(f'fc{i+1}', nn.Linear(30, self.n_mid))
      else:
        self.layers.add_module(f'fc{i+1}', nn.Linear(self.n_mid, self.n_mid))
      self.layers.add_module(f'act{i+1}', nn.ReLU())
      self.layers.add_module(f'bt{i+1}', nn.BatchNorm1d(self.n_mid))
    
    self.layers.add_module(f'fc{self.n_layers+1}', nn.Linear(self.n_mid, 2))
      
  def forward(self, x):
    h = self.layers(x)
    h = F.softmax(h)
    return h

  def training_step(self, batch, batch_idx):
    x, t=batch
    y = self.forward(x)
    loss = F.cross_entropy(y, t)
    return loss

  def validation_step(self, batch, batch_idx):
    x, t=batch
    y = self.forward(x)
    loss = F.cross_entropy(y, t)
    acc = accuracy(y, t)
    self.log('val_loss', loss, on_epoch=True)
    self.log('val_acc', acc, on_epoch=True)
    return loss

  def test_step(self, batch, batch_idx):
    x, t=batch
    y = self.forward(x)
    loss = F.cross_entropy(y, t)
    self.log('test_loss', loss, on_epoch=True)
    self.log('test_acc', acc, on_epoch=True)
    return loss

  def configure_optimizers(self):
    #learning rateの探索
    optimizer = torch.optim.SGD(self.parameters(), lr=self.lr)
    return optimizer

In [54]:
net = Net(n_layers=4)
net

Net(
  (layers): Sequential(
    (fc1): Linear(in_features=30, out_features=10, bias=True)
    (act1): ReLU()
    (bt1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc2): Linear(in_features=10, out_features=10, bias=True)
    (act2): ReLU()
    (bt2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc3): Linear(in_features=10, out_features=10, bias=True)
    (act3): ReLU()
    (bt3): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc4): Linear(in_features=10, out_features=10, bias=True)
    (act4): ReLU()
    (bt4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc5): Linear(in_features=10, out_features=2, bias=True)
  )
)

In [19]:
net = Net()
pl.seed_everything(0)
trainer = pl.Trainer(max_epochs=30, gpus=1, deterministic=True)
trainer.fit(net, train_loader, val_loader)

Global seed set to 0
GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 310   
1 | fc2  | Linear | 22    
--------------------------------
332       Trainable params
0         Non-trainable params
332       Total params
0.001     Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning:

Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



1

In [20]:
trainer.callback_metrics

{'val_acc': tensor(0.6771, device='cuda:0'),
 'val_loss': tensor(0.6362, device='cuda:0')}

### ハイパーパラメーターの範囲指定と、目的関数をもった関数の作成

In [57]:
def objctive(trial):
  #lerning rate
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
  #node count 
  n_mid = trial.suggest_int('n_mid', 2, 100)

  n_layers = trial.suggest_int('n_layers', 1, 5)

  #目的関数：検証データのAccuracy
  pl.seed_everything(0)
  net = Net(lr=lr, n_mid=n_mid, n_layers=n_layers)
  trainer = pl.Trainer(max_epochs=30, gpus=1, deterministic=True, callbacks=[EarlyStopping(monitor='val_acc')])
  trainer.fit(net, train_dataloader=train_loader, val_dataloaders=val_loader)

  return trainer.callback_metrics['val_acc']

In [58]:
sampler = optuna.samplers.TPESampler(seed=0)
study = optuna.create_study(sampler=sampler, direction='maximize') #val_accの最大化が目的
study.optimize(objctive, n_trials=30) #30回試行

[I 2021-03-15 09:40:20,344] A new study created in memory with name: no-name-76c52041-79c2-40e2-8fed-d9cbf5ff08eb
Global seed set to 0
GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 16.0 K
--------------------------------------
16.0 K    Trainable params
0         Non-trainable params
16.0 K    Total params
0.064     Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning:

Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



[I 2021-03-15 09:40:22,641] Trial 0 finished with value: 0.8333333134651184 and parameters: {'lr': 0.0015676677195506068, 'n_mid': 66, 'n_layers': 4}. Best is trial 0 with value: 0.8333333134651184.
Global seed set to 0
GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 849   
--------------------------------------
849       Trainable params
0         Non-trainable params
849       Total params
0.003     Total estimated model params size (MB)


[I 2021-03-15 09:40:24,523] Trial 1 finished with value: 0.3541666567325592 and parameters: {'lr': 0.027026043440193118, 'n_mid': 11, 'n_layers': 4}. Best is trial 0 with value: 0.8333333134651184.
Global seed set to 0


GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 1.3 K 
--------------------------------------
1.3 K     Trainable params
0         Non-trainable params
1.3 K     Total params
0.005     Total estimated model params size (MB)


...

[I 2021-03-15 09:41:18,137] Trial 25 finished with value: 0.9479166865348816 and parameters: {'lr': 0.009024256044285297, 'n_mid': 18, 'n_layers': 2}. Best is trial 2 with value: 0.9479166865348816.
Global seed set to 0
GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 968   
--------------------------------------
968       Trainable params
0         Non-trainable params
968       Total params
0.004     Total estimated model params size (MB)


[I 2021-03-15 09:41:20,431] Trial 26 finished with value: 0.8854166865348816 and parameters: {'lr': 0.0026353909708405287, 'n_mid': 14, 'n_layers': 3}. Best is trial 2 with value: 0.9479166865348816.
Global seed set to 0
GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 2.0 K 
--------------------------------------
2.0 K     Trainable params
0         Non-trainable params
2.0 K     Total params
0.008     Total estimated model params size (MB)


[I 2021-03-15 09:41:23,426] Trial 27 finished with value: 0.8229166865348816 and parameters: {'lr': 0.000779391175939973, 'n_mid': 30, 'n_layers': 2}. Best is trial 2 with value: 0.9479166865348816.
Global seed set to 0
GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 82    
--------------------------------------
82        Trainable params
0         Non-trainable params
82        Total params
0.000     Total estimated model params size (MB)


[I 2021-03-15 09:41:24,668] Trial 28 finished with value: 0.8645833134651184 and parameters: {'lr': 0.006679933852163009, 'n_mid': 2, 'n_layers': 2}. Best is trial 2 with value: 0.9479166865348816.
Global seed set to 0
GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 782   
--------------------------------------
782       Trainable params
0         Non-trainable params
782       Total params
0.003     Total estimated model params size (MB)


[I 2021-03-15 09:41:25,890] Trial 29 finished with value: 0.65625 and parameters: {'lr': 0.0016002511012477363, 'n_mid': 12, 'n_layers': 3}. Best is trial 2 with value: 0.9479166865348816.


In [59]:
study.best_value, study.best_params

(0.9479166865348816, {'lr': 0.0038333321561566623, 'n_layers': 1, 'n_mid': 38})

In [60]:
study.best_trial

FrozenTrial(number=2, values=[0.9479166865348816], datetime_start=datetime.datetime(2021, 3, 15, 9, 40, 24, 525052), datetime_complete=datetime.datetime(2021, 3, 15, 9, 40, 26, 566289), params={'lr': 0.0038333321561566623, 'n_mid': 38, 'n_layers': 1}, distributions={'lr': LogUniformDistribution(high=0.1, low=1e-05), 'n_mid': IntUniformDistribution(high=100, low=2, step=1), 'n_layers': IntUniformDistribution(high=5, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=2, state=TrialState.COMPLETE, value=None)

Trial数による目的関数の動きの可視化

In [61]:
optuna.visualization.plot_optimization_history(study)

In [35]:
optuna.visualization.plot_slice(study)

n_mid,lrの関係性をplot  
**lr=0.01の範囲で、中間ノード数は大きい方が結果としては良いことが言える。**  
**Layerの影響はあまりなし。**

In [62]:
optuna.visualization.plot_contour(study)

In [47]:
layers = nn.Sequential()

for n in range(5):
  layers.add_module(f'fc{n+1}', nn.Linear(30, 30))
  layers.add_module(f'act{n+1}', nn.ReLU())
  layers.add_module(f'bn{n+1}', nn.BatchNorm1d(30))

layers

Sequential(
  (fc1): Linear(in_features=30, out_features=30, bias=True)
  (act1): ReLU()
  (bn1): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=30, out_features=30, bias=True)
  (act2): ReLU()
  (bn2): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=30, out_features=30, bias=True)
  (act3): ReLU()
  (bn3): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=30, out_features=30, bias=True)
  (act4): ReLU()
  (bn4): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc5): Linear(in_features=30, out_features=30, bias=True)
  (act5): ReLU()
  (bn5): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)